In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import requests
from bs4 import BeautifulSoup

# Path to the uploaded `unique_urls.txt` file
urls_file_path = '/content/drive/MyDrive/unique_urls.txt'

# Load the URLs from the file
with open(urls_file_path, 'r') as file:
    urls = file.read().splitlines()

scraped_content = {}

# Function to scrape a single URL and extract paragraphs
def scrape_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract paragraphs
        paragraphs = soup.find_all('p')
        text = ' '.join(paragraph.get_text() for paragraph in paragraphs)

        if text.strip():  # Check if the text is not empty
            return text
        else:
            print(f"Content from {url} is empty or only contains non-paragraph elements.")
            return None
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")
        return None

# Scrape each URL
for url in urls:
    content = scrape_url(url)
    if content:
        scraped_content[url] = content

# Save the scraped content to a file in Google Drive
output_file_path = '/content/drive/MyDrive/scrap_content.txt'
with open(output_file_path, 'w') as file:
    for url, content in scraped_content.items():
        file.write(f"URL: {url}\n")
        file.write(content + "\n\n")

print(f"Web scraping completed and content saved to '{output_file_path}'.")


Failed to scrape Diamond Model of Intrusion Analysis_part0.txt: Invalid URL 'Diamond Model of Intrusion Analysis_part0.txt': No scheme supplied. Perhaps you meant https://Diamond Model of Intrusion Analysis_part0.txt?
Failed to scrape Diamond Model of Intrusion Analysis_part1.txt: Invalid URL 'Diamond Model of Intrusion Analysis_part1.txt': No scheme supplied. Perhaps you meant https://Diamond Model of Intrusion Analysis_part1.txt?
Failed to scrape Diamond Model of Intrusion Analysis_part2.txt: Invalid URL 'Diamond Model of Intrusion Analysis_part2.txt': No scheme supplied. Perhaps you meant https://Diamond Model of Intrusion Analysis_part2.txt?
Failed to scrape Diamond Model of Intrusion Analysis_part3.txt: Invalid URL 'Diamond Model of Intrusion Analysis_part3.txt': No scheme supplied. Perhaps you meant https://Diamond Model of Intrusion Analysis_part3.txt?
Failed to scrape Diamond Model of Intrusion Analysis_part4.txt: Invalid URL 'Diamond Model of Intrusion Analysis_part4.txt': No 

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 92.9 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF
import os
import re

# Path to the directory containing PDF files
input_directory = '/content/drive/MyDrive/pdfs'

# Path to the directory where text files will be saved
output_directory = '/content/drive/MyDrive/processed_pdf'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text()
        doc.close()
    except Exception as e:
        print(f"Failed to extract text from {pdf_path}: {e}")
    return text

# Function to filter out non-relevant sections
def filter_text(text):
    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)

    # Remove lines with table of contents or references
    text = re.sub(r'(Table of Contents|References)', '', text, flags=re.IGNORECASE)

    # Remove introductory or presentation text (basic example)
    introduction_patterns = [
        r'\bintroduction\b',
        r'\boverview\b',
        r'\bpresentation\b',
        r'\bsummary\b'
    ]

    for pattern in introduction_patterns:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    # Normalize multiple newlines
    text = re.sub(r'\n\s*\n+', '\n\n', text)

    # Optional: remove lines that are just titles or headers
    text = '\n'.join(line for line in text.split('\n') if not line.isupper() and len(line) > 2)

    return text

# Function to divide text into parts based on length
def divide_text_into_parts(text, part_length=7000):
    parts = []
    while len(text) > part_length:
        # Find a reasonable split point (last newline before reaching part_length)
        split_point = text.rfind('\n', 0, part_length)
        if split_point == -1:
            split_point = part_length
        part = text[:split_point].strip()
        parts.append(part)
        text = text[split_point:].strip()
    if text:
        parts.append(text)
    return parts

# List all PDF files in the input directory
files = [f for f in os.listdir(input_directory) if f.endswith('.pdf')]

# Process each PDF file
for filename in files:
    pdf_path = os.path.join(input_directory, filename)
    try:
        text = extract_text_from_pdf(pdf_path)

        # Filter out non-relevant sections
        filtered_text = filter_text(text)

        # Divide text into parts
        parts = divide_text_into_parts(filtered_text)

        # Save each part into separate text files in the output directory
        for i, part in enumerate(parts):
            part_filename = f"{os.path.splitext(filename)[0]}_part{i}.txt"
            part_path = os.path.join(output_directory, part_filename)
            with open(part_path, 'w', encoding='utf-8') as file:
                file.write(part)

            print(f"Saved part {i+1} of '{filename}' to '{part_path}'")

    except Exception as e:
        print(f"Failed to process {filename}: {e}")

print("Text extraction, filtering, and division completed.")


Saved part 1 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part0.txt'
Saved part 2 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part1.txt'
Saved part 3 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part2.txt'
Saved part 4 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part3.txt'
Saved part 5 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part4.txt'
Saved part 6 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part5.txt'
Saved part 7 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part6.txt'
Saved part 8 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part7.txt'
Saved part 9 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part8.txt'
Saved part 10 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part9.txt'
Saved part 11 of 'diamond.pdf' to '/content/drive/MyDrive/processed_pdf/diamond_part10.txt'
Saved p

qcm generation


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
api_key = os.getenv('HF_TOKEN')

# Load the tokenizer and model from Hugging Face
model_name = "mistralai/Mistral-7B-Instruct-v0.3"  # Update this with the correct Mistral model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


def list_files_in_directory(directory):
    """Lists all files in a given directory."""
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

def read_text_file(file_path):
    """Reads text from a specified file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def chunk_text(text, chunk_size=2048):
    """Splits text into manageable chunks."""
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield ' '.join(words[i:i + chunk_size])

def generate_mcqs_mistral(chunk, num_questions=5):
    """Generates MCQs using the Mistral model."""
    prompt = (
        "You are a cybersecurity expert specializing in cyber threat intelligence., please "
        f"Generate a maximum of {num_questions} concept-based multiple-choice questions "
        "with four options each for a cybersecurity audience. Follow these requirements:\n"
        "1. Questions must cover advanced concepts suitable for professionals with 3-5 years of experience.\n"
        "2. Avoid generic questions; focus on context-specific concepts.\n"
        "3. Incorporate situational or scenario-based questions where applicable.\n"
        "4. Use precise cybersecurity terminology.\n"
        "5. Mention MITRE ATT&CK techniques where relevant, including IDs and names.\n"
        "6. Output format: TSV format with columns: Question, Option A, Option B, Option C, Option D, Correct Answer, Explanation.\n"
        f"Text:\n{chunk}\n\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt", max_length=4096, truncation=True)
    outputs = model.generate(**inputs, max_length=4096, num_return_sequences=1, do_sample=True, temperature=0.7)
    mcq_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return mcq_text

def save_mcqs_to_tsv(mcqs_list, base_output_file):
    """Saves MCQs to TSV files, splitting into multiple files if necessary."""
    num_files = (len(mcqs_list) // 500) + 1  # Save 500 MCQs per file
    for i in range(num_files):
        start_idx = i * 500
        end_idx = min((i + 1) * 500, len(mcqs_list))
        mcqs_chunk = mcqs_list[start_idx:end_idx]
        output_file = f"{base_output_file}_{i+1}.tsv"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(mcqs_chunk))
        print(f"MCQs have been saved to {output_file}")

# Specify the directory path
directory_path = '/content/drive/MyDrive/processed_pdf'

# List files in the directory to identify text files
files = list_files_in_directory(directory_path)
print("Files in directory:", files)

# Process each text file in the directory
for file_name in files:
    file_path = os.path.join(directory_path, file_name)

    # Read text from the selected text file
    text = read_text_file(file_path)

    # Generate MCQs
    mcqs_list = []
    for chunk in chunk_text(text):
        mcqs = generate_mcqs_mistral(chunk, num_questions=10)
        mcqs_list.append(mcqs)

    # Save the MCQs to TSV files
    base_output_file = os.path.splitext(file_name)[0]  # Use the file name without extension
    save_mcqs_to_tsv(mcqs_list, base_output_file)


Files in directory: ['GDPR_FINAL_EPSU_part3.txt', 'GDPR_FINAL_EPSU_part4.txt', 'GDPR_FINAL_EPSU_part5.txt', 'GDPR_FINAL_EPSU_part6.txt', 'GDPR_FINAL_EPSU_part7.txt', 'GDPR_FINAL_EPSU_part9.txt']


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MCQs have been saved to GDPR_FINAL_EPSU_part3_1.tsv


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MCQs have been saved to GDPR_FINAL_EPSU_part4_1.tsv


...

MCQs have been saved to GDPR_FINAL_EPSU_part5_1.tsv


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MCQs have been saved to GDPR_FINAL_EPSU_part6_1.tsv


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def list_files_in_directory(directory):
    """Lists all files in a given directory."""
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

def read_text_file(file_path):
    """Reads text from a specified file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def chunk_text(text, chunk_size=2048):
    """Splits text into manageable chunks."""
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield ' '.join(words[i:i + chunk_size])

def generate_mcqs_mistral(chunk, num_questions=5):
    """Generates MCQs using the Mistral model."""
    prompt = (
        "You are a cybersecurity expert specializing in cyber threat intelligence., please "
        f"Generate a maximum of {num_questions} concept-based multiple-choice questions "
        "with four options each for a cybersecurity audience. Follow these requirements:\n"
        "1. Questions must cover advanced concepts suitable for professionals with 3-5 years of experience.\n"
        "2. Avoid generic questions; focus on context-specific concepts.\n"
        "3. Incorporate situational or scenario-based questions where applicable.\n"
        "4. Use precise cybersecurity terminology.\n"
        "5. Mention MITRE ATT&CK techniques where relevant, including IDs and names.\n"
        "6. Output format: TSV format with columns: Question, Option A, Option B, Option C, Option D, Correct Answer, Explanation.\n"
        f"Text:\n{chunk}\n\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt", max_length=4096, truncation=True)
    outputs = model.generate(**inputs, max_length=8096, num_return_sequences=1, do_sample=True, temperature=0.7)
    mcq_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return mcq_text


def save_mcqs_to_tsv(mcqs_list, base_output_file):
    """Saves MCQs to TSV files, splitting into multiple files if necessary."""
    num_files = (len(mcqs_list) // 500) + 1  # Save 500 MCQs per file
    for i in range(num_files):
        start_idx = i * 500
        end_idx = min((i + 1) * 500, len(mcqs_list))
        mcqs_chunk = mcqs_list[start_idx:end_idx]

        # Save the files to the new output directory
        output_directory = '/content/drive/MyDrive/mm/qcm_cti'
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        output_file = os.path.join(output_directory, f"{base_output_file}_{i+1}.tsv")

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(mcqs_chunk))
        print(f"MCQs have been saved to {output_file}")


# Specify the directory path
directory_path = '/content/drive/MyDrive/processed_pdf'

# List files in the directory to identify text files
files = list_files_in_directory(directory_path)
print("Files in directory:", files)

# Process each text file in the directory
for file_name in files:
    file_path = os.path.join(directory_path, file_name)

    # Read text from the selected text file
    text = read_text_file(file_path)

    # Generate MCQs
    mcqs_list = []
    for chunk in chunk_text(text):
        mcqs = generate_mcqs_mistral(chunk, num_questions=30)
        mcqs_list.append(mcqs)

    # Save the MCQs to TSV files
    base_output_file = os.path.splitext(file_name)[0]  # Use the file name without extension
    save_mcqs_to_tsv(mcqs_list, base_output_file)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Files in directory: ['GDPR_FINAL_EPSU_part3.txt', 'GDPR_FINAL_EPSU_part4.txt', 'GDPR_FINAL_EPSU_part5.txt', 'GDPR_FINAL_EPSU_part6.txt', 'GDPR_FINAL_EPSU_part7.txt', 'GDPR_FINAL_EPSU_part9.txt']


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MCQs have been saved to /content/drive/MyDrive/mm/qcm_cti/GDPR_FINAL_EPSU_part3_1.tsv


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

MCQs have been saved to /content/drive/MyDrive/mm/qcm_cti/GDPR_FINAL_EPSU_part5_1.tsv


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MCQs have been saved to /content/drive/MyDrive/mm/qcm_cti/GDPR_FINAL_EPSU_part6_1.tsv
MCQs have been saved to /content/drive/MyDrive/mm/qcm_cti/GDPR_FINAL_EPSU_part7_1.tsv


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MCQs have been saved to /content/drive/MyDrive/mm/qcm_cti/GDPR_FINAL_EPSU_part9_1.tsv


In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

def list_files_in_directory(directory):
    """Lists all files in a given directory."""
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

def read_text_file(file_path):
    """Reads text from a specified file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def chunk_text(text, chunk_size=2048):
    """Splits text into manageable chunks."""
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield ' '.join(words[i:i + chunk_size])

def generate_mcqs_mistral(chunk, num_questions=5):
    """Generates MCQs using the Mistral model."""
    prompt = (
        "You are a cybersecurity expert specializing in cyber threat intelligence. Please "
        f"Generate a maximum of {num_questions} concept-based multiple-choice questions "
        "with four options each for a cybersecurity audience. Follow these requirements:\n"
        "1. Questions must cover advanced concepts suitable for professionals with 3-5 years of experience.\n"
        "2. Avoid generic questions; focus on context-specific concepts.\n"
        "3. Incorporate situational or scenario-based questions where applicable.\n"
        "4. Use precise cybersecurity terminology.\n"
        "5. Mention MITRE ATT&CK techniques where relevant, including IDs and names.\n"
        "6. Output format: TSV format with columns: Question, Option A, Option B, Option C, Option D, Correct Answer, Explanation.\n"
        f"Text:\n{chunk}\n\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt", max_length=4096, truncation=True)
    outputs = model.generate(**inputs, max_length=4096, num_return_sequences=1, do_sample=True, temperature=0.7)
    mcq_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return mcq_text

def save_mcqs_to_tsv(mcqs_list, base_output_file):
    """Saves MCQs to TSV files, splitting into multiple files if necessary."""
    num_files = (len(mcqs_list) // 500) + 1  # Save 500 MCQs per file
    for i in range(num_files):
        start_idx = i * 500
        end_idx = min((i + 1) * 500, len(mcqs_list))
        mcqs_chunk = mcqs_list[start_idx:end_idx]
        output_file = f"{base_output_file}_{i+1}.tsv"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(mcqs_chunk))
        print(f"MCQs have been saved to {output_file}")

# Load model and tokenizer
model_name = 'mistral/your-model-name'  # Replace with your model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Specify the directory path
directory_path = '/content/drive/MyDrive/processed_pdf'

# List files in the directory to identify text files
files = list_files_in_directory(directory_path)
print("Files in directory:", files)

# Process each text file in the directory
for file_name in files:
    file_path = os.path.join(directory_path, file_name)

    # Read text from the selected text file
    text = read_text_file(file_path)

    # Generate MCQs
    mcqs_list = []
    for chunk in chunk_text(text):
        mcqs = generate_mcqs_mistral(chunk, num_questions=10)
        mcqs_list.append(mcqs)

    # Save the MCQs to TSV files
    base_output_file = os.path.splitext(file_name)[0]  # Use the file name without extension
    save_mcqs_to_tsv(mcqs_list, base_output_file)


In [ ]:
import os

def list_files_in_directory(directory):
    """Lists all files in a given directory."""
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# Specify the directory path
directory_path = '/content/drive/MyDrive/mm/qcm_cti'

# List files in the directory
files = list_files_in_directory(directory_path)

# Print the file names
for file_name in files:
    print(file_name)


NIST.SP.800-150_part2_1.tsv
NIST.SP.800-150_part3_1.tsv
NIST.SP.800-150_part4_1.tsv
NIST.SP.800-150_part5_1.tsv
NIST.SP.800-150_part6_1.tsv
NIST.SP.800-150_part7_1.tsv
NIST.SP.800-150_part8_1.tsv
NIST.SP.800-150_part9_1.tsv
NIST.SP.800-150_part10_1.tsv
NIST.SP.800-150_part11_1.tsv
NIST.SP.800-150_part12_1.tsv
NIST.SP.800-150_part13_1.tsv
NIST.SP.800-150_part14_1.tsv
NIST.SP.800-150_part15_1.tsv
NIST.SP.800-150_part16_1.tsv
NIST.SP.800-150_part17_1.tsv
diamond_part0_1.tsv
diamond_part1_1.tsv
diamond_part2_1.tsv
diamond_part3_1.tsv
diamond_part4_1.tsv
diamond_part5_1.tsv
diamond_part6_1.tsv
diamond_part7_1.tsv
diamond_part8_1.tsv
diamond_part9_1.tsv
diamond_part10_1.tsv
diamond_part11_1.tsv
diamond_part12_1.tsv
diamond_part13_1.tsv
diamond_part14_1.tsv
diamond_part16_1.tsv
diamond_part17_1.tsv
diamond_part18_1.tsv
NIST.CSWP.29_part0_1.tsv
NIST.CSWP.29_part1_1.tsv
NIST.CSWP.29_part2_1.tsv
NIST.CSWP.29_part3_1.tsv
NIST.CSWP.29_part4_1.tsv
NIST.CSWP.29_part8_1.tsv
NIST.CSWP.29_part5_1.tsv
N

In [ ]:
import shutil
import os

def copy_directory(src, dst):
    """Copies the directory from src to dst."""
    if os.path.exists(dst):
        shutil.rmtree(dst)  # Remove existing directory
    shutil.copytree(src, dst)  # Copy the source directory
    print(f"Directory copied from {src} to {dst}")

def filter_files(src, dst, keep_files):
    """Copies only specified files from src to dst."""
    if not os.path.exists(dst):
        os.makedirs(dst)  # Create the destination directory if it doesn't exist

    for file_name in os.listdir(src):
        file_path = os.path.join(src, file_name)
        if file_name in keep_files and os.path.isfile(file_path):
            shutil.copy(file_path, os.path.join(dst, file_name))
            print(f"Copied {file_name} to {dst}")
        elif os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Removed {file_name} from {src}")

def main():
    # Define paths
    original_directory_path = '/content/drive/MyDrive/processed_pdf_copy'
    copy_directory_path = '/content/drive/MyDrive/processed_pdf_filtered'

    # List of files to keep
    keep_files = [
        'NIST.SP.800-150_part3.txt', 'NIST.SP.800-150_part4.txt', 'NIST.SP.800-150_part5.txt',
        'NIST.SP.800-150_part6.txt', 'NIST.SP.800-150_part7.txt', 'NIST.SP.800-150_part8.txt',
        'NIST.SP.800-150_part9.txt', 'NIST.SP.800-150_part10.txt', 'NIST.SP.800-150_part11.txt',
        'NIST.SP.800-150_part12.txt', 'NIST.SP.800-150_part13.txt', 'NIST.SP.800-150_part14.txt',
        'NIST.SP.800-150_part15.txt', 'NIST.SP.800-150_part16.txt', 'NIST.SP.800-150_part17.txt',
        'diamond_part0.txt', 'diamond_part1.txt', 'diamond_part2.txt', 'diamond_part3.txt',
        'diamond_part4.txt', 'diamond_part5.txt', 'diamond_part6.txt', 'diamond_part7.txt',
        'diamond_part8.txt', 'diamond_part9.txt', 'diamond_part10.txt', 'diamond_part11.txt',
        'diamond_part12.txt', 'diamond_part13.txt', 'diamond_part14.txt', 'diamond_part15.txt',
        'diamond_part16.txt', 'diamond_part17.txt', 'diamond_part18.txt', 'NIST.CSWP.29_part0.txt',
        'NIST.CSWP.29_part1.txt', 'NIST.CSWP.29_part2.txt', 'NIST.CSWP.29_part3.txt',
        'NIST.CSWP.29_part4.txt', 'NIST.CSWP.29_part5.txt', 'NIST.CSWP.29_part6.txt',
        'NIST.CSWP.29_part7.txt', 'NIST.CSWP.29_part8.txt', 'NIST.CSWP.29_part9.txt',
        'stix-intro-handout_part0.txt', 'taxii-intro-handout_part0.txt', 'taxii-intro-handout_part1.txt',
        'A Framework for Cyber Threat Hunting Part 1_ The Pyramid of Pain _ Sqrrl_part0.txt',
        'NIST.SP.800-61r2_part0.txt', 'NIST.SP.800-61r2_part1.txt', 'NIST.SP.800-61r2_part2.txt',
        'NIST.SP.800-61r2_part3.txt', 'NIST.SP.800-61r2_part4.txt', 'NIST.SP.800-61r2_part5.txt',
        'NIST.SP.800-61r2_part6.txt', 'NIST.SP.800-61r2_part7.txt', 'NIST.SP.800-61r2_part8.txt',
        'NIST.SP.800-61r2_part9.txt', 'NIST.SP.800-61r2_part10.txt', 'NIST.SP.800-61r2_part11.txt',
        'NIST.SP.800-61r2_part12.txt', 'NIST.SP.800-61r2_part13.txt', 'NIST.SP.800-61r2_part14.txt',
        'NIST.SP.800-61r2_part15.txt', 'NIST.SP.800-61r2_part16.txt', 'NIST.SP.800-61r2_part17.txt',
        'NIST.SP.800-61r2_part18.txt', 'NIST.SP.800-61r2_part19.txt', 'NIST.SP.800-61r2_part20.txt',
        'NIST.SP.800-61r2_part21.txt', 'NIST.SP.800-61r2_part22.txt', 'NIST.SP.800-61r2_part23.txt',
        'NIST.SP.800-61r2_part24.txt', 'NIST.SP.800-61r2_part25.txt', 'NIST.SP.800-61r2_part26.txt',
        'NIST.SP.800-61r2_part27.txt', 'NIST.SP.800-61r2_part28.txt', 'NIST.SP.800-61r2_part29.txt',
        'NIST.SP.800-61r2_part30.txt', 'NIST.SP.800-61r2_part31.txt', 'GDPR_FINAL_EPSU_part0.txt',
        'GDPR_FINAL_EPSU_part1.txt', 'GDPR_FINAL_EPSU_part2.txt', 'GDPR_FINAL_EPSU_part3.txt',
        'GDPR_FINAL_EPSU_part4.txt', 'GDPR_FINAL_EPSU_part5.txt', 'GDPR_FINAL_EPSU_part6.txt',
        'GDPR_FINAL_EPSU_part7.txt', 'GDPR_FINAL_EPSU_part8.txt', 'GDPR_FINAL_EPSU_part9.txt',
        'NIST.SP.800-150_part1.txt', 'NIST.SP.800-150_part2.txt'
    ]

    # Copy the directory
    copy_directory(original_directory_path, copy_directory_path)

    # Filter files to keep only specified ones
    filter_files(original_directory_path, copy_directory_path, keep_files)

if __name__ == "__main__":
    main()


Directory copied from /content/drive/MyDrive/processed_pdf_copy to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part3.txt to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part4.txt to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part5.txt to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part6.txt to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part7.txt to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part8.txt to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part9.txt to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part10.txt to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part11.txt to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part12.txt to /content/drive/MyDrive/processed_pdf_filtered
Copied NIST.SP.800-150_part13.txt to /content/drive/MyDrive/proc

                                                              processing data

gemini


In [ ]:
!pip install google-generativeai


  Using cached google_generativeai-0.7.2-py3-none-any.whl.metadata (4.0 kB)
Using cached google_generativeai-0.7.2-py3-none-any.whl (164 kB)


In [ ]:
import google.generativeai as genai
from google.colab import userdata
gemini_key = userdata.get('gemini_key')


genai.configure(api_key=gemini_key)

In [ ]:
# configuration du modèle
generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 5,
  "max_output_tokens": 2048,
}

**Processing my database with gemini **

In [ ]:
import pandas as pd
import re
import google.generativeai as genai
import time

# Load the dataset from the TSV file
df = pd.read_csv('/content/drive/MyDrive/myDB.tsv', sep='\t', on_bad_lines='warn')

# Trim any leading/trailing whitespace from column names
df.columns = df.columns.str.strip()

# Drop unnecessary columns, if they exist
df = df.drop(columns=['Explanation', 'Correct_Answer'], errors='ignore')

# Function to prepare the prompt for the model
def prepare_prompt(question, options):
    options_text = "\n".join([f"Option {chr(65+i)}: {opt}" for i, opt in enumerate(options)])
    formatted_prompt = (
        "You are a cybersecurity expert specializing in cyber threat intelligence. "
        "You are given a multiple-choice question (MCQ) from a Cyber Threat Intelligence (CTI) knowledge benchmark dataset. "
        "Your task is to choose the best option among the four provided. Return your answer as a single uppercase letter: A, B, C, or D.\n"
        f"**Question:** {question}\n**Options:**\n{options_text}\n**Important:** The last line of your answer should contain only the single letter corresponding to the best option, with no additional text.\n"
        "**Answer:**"
    )
    return formatted_prompt

# Function to generate response from the Gemini model
def generate_response(prompt, retries=5, delay=20):
    for attempt in range(retries):
        try:
            response = genai.chat(prompt=prompt)
            if response.candidates and 'content' in response.candidates[0]:
                return response.candidates[0]['content'].strip()
            else:
                raise ValueError("Invalid response structure.")
        except Exception as e:
            if attempt < retries - 1:
                print(f"Error: {e}. Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"Failed after {retries} attempts.")
                return 'ERROR'

# Function to extract the answer from generated text
def extract_answer(generated_text):
    # Find the last line that contains only a single uppercase letter
    match = re.findall(r'\b[A-D]\b', generated_text)
    return match[-1] if match else "No valid answer found"

# Process the questions and generate responses
results = []
num_processed = 0
for idx, row in df.iterrows():
    if idx >= 100:
        break

    print(f"Processing question {idx + 1}...")  # Debug print to track progress

    question = row['question']
    options = [row['Option_A'], row['Option_B'], row['Option_C'], row['Option_D']]

    # Prepare the prompt
    formatted_prompt = prepare_prompt(question, options)

    # Generate a response
    generated_text = generate_response(formatted_prompt)

    # Extract the answer
    predicted_answer = extract_answer(generated_text)

    # Append results
    results.append({
        'Question': question,
        'Formatted Prompt': formatted_prompt,
        'Generated Text': generated_text,
        'Predicted Answer': predicted_answer,
    })

    num_processed += 1
    # Save results incrementally
    results_df = pd.DataFrame(results)
    results_df.to_csv('/content/drive/MyDrive/mydb_answer_GEMINI.csv', index=False)

    print(f"Processed {num_processed} out of 100 questions")  # Debug print to track progress

print(f"Processed {num_processed} questions")
print("Predicted answers saved to '/content/drive/MyDrive/mydb_answer_GEMINI.csv'")


Processing question 1...
Processed 1 out of 100 questions
Processing question 2...
Processed 2 out of 100 questions
Processing question 3...
Processed 3 out of 100 questions
Processing question 4...
Processed 4 out of 100 questions
Processing question 5...
Processed 5 out of 100 questions
Processing question 6...
Processed 6 out of 100 questions
Processing question 7...
Processed 7 out of 100 questions
Processing question 8...
Processed 8 out of 100 questions
Processing question 9...
Processed 9 out of 100 questions
Processing question 10...
Processed 10 out of 100 questions
Processing question 11...
Processed 11 out of 100 questions
Processing question 12...
Processed 12 out of 100 questions
Processing question 13...
Processed 13 out of 100 questions
Processing question 14...
Processed 14 out of 100 questions
Processing question 15...
Processed 15 out of 100 questions
Processing question 16...
Processed 16 out of 100 questions
Processing question 17...
Processed 17 out of 100 question

In [ ]:
import pandas as pd
import re

# Define the file path
file_path = '/content/drive/MyDrive/cti-mcq-gemini.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Define a function to extract the correct answer from the Model Response
def extract_correct_answer(model_response):
    """Extracts the correct answer from the model response."""
    patterns = [
        r'THE CORRECT ANSWER IS:?\s*[*]*([A-D])[*)]*',
        r'THE ANSWER IS:?\s*[*]*([A-D])[*)]*',
        r'CORRECT ANSWER:?\s*[*]*([A-D])[*)]*',
        r'ANSWER:?\s*[*]*([A-D])[*)]*',
        r'CORRECT:\s*[*]*([A-D])[*)]*',
        r'ANSWER:\s*([A-D])',
        r'CORRECT:\s*([A-D])',
        r'\b([A-D])\b'
    ]

    # Check each pattern
    for pattern in patterns:
        matches = re.findall(pattern, model_response, re.IGNORECASE)
        if matches:
            # Take the last match as the correct answer
            return matches[-1]

    # Debugging output to help identify the issue
    print(f"DEBUG: No match found for Model Response:\n{model_response}\n")
    return None

# Function to compare and update the 'Is Correct' column
def update_is_correct(row):
    model_answer = row['Model Answer'].strip().upper()
    correct_answer = row['Correct Answer'].strip().upper()
    if model_answer == correct_answer:
        return True
    else:
        return False

# Extract correct answers from the 'Model Response' column
df['Model Answer'] = df.apply(lambda row: extract_correct_answer(row['Model Response']) if row['Model Answer'].strip().upper() == 'T' else row['Model Answer'], axis=1)

# Update the 'Is Correct' column
df['Is Correct'] = df.apply(update_is_correct, axis=1)

# Save the modified DataFrame back to a CSV file
output_file_path = '/content/drive/MyDrive/cti-mcq-gemini-updated.csv'
df.to_csv(output_file_path, index=False)

print(f"Updated file saved to {output_file_path}")


Updated file saved to /content/drive/MyDrive/cti-mcq-gemini-updated.csv


In [ ]:
import pandas as pd

# Define the file path
file_path = '/content/drive/MyDrive/cti-mcq-gemini-updated.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Calculate the accuracy
total_questions = len(df)
correct_answers = df['Is Correct'].sum()  # Sum of True values in 'Is Correct'

accuracy = (correct_answers / total_questions) * 100

print(f"Total Questions: {total_questions}")
print(f"Correct Answers: {correct_answers}")
print(f"Accuracy: {accuracy:.2f}%")


Total Questions: 100
Correct Answers: 61
Accuracy: 61.00%


***gemini evaluation***


*   Accuracy: 61.00%




In [ ]:
import pandas as pd
import re

# Define the file path
file_path = '/content/drive/MyDrive/mydb_answer_GEMINI.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Function to extract answer from generated text
def extract_answer(text):
    # Regular expression patterns for various formats
    patterns = [
        r'\bTHE CORRECT ANSWER IS: ([A-D])\b',
        r'\bTHE ANSWER IS: ([A-D])\b',
        r'\bThe correct answer is: ([A-D])\b',
        r'\bThe answer is ([A-D])\b',
        r'\b([A-D])\b'  # For cases where only the letter might be present
    ]

    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(1)

    return None

# Update the Predicted Answer column based on the Generated Text
df['Predicted Answer'] = df['Generated Text'].apply(extract_answer)

# Save the updated DataFrame back to a CSV file
output_file_path = '/content/drive/MyDrive/mydb_answer_GEMINI_updated.csv'
df.to_csv(output_file_path, index=False)

print(f"Updated file saved to: {output_file_path}")


Updated file saved to: /content/drive/MyDrive/mydb_answer_GEMINI_updated.csv


In [ ]:
import pandas as pd

# Define file paths
predicted_file_path = '/content/drive/MyDrive/mydb_answer_GEMINI_updated.csv'
correct_answers_file_path = '/content/drive/MyDrive/myDB.tsv'
output_file_path = '/content/drive/MyDrive/combined_results.csv'

# Load the CSV file with predicted answers
df_predicted = pd.read_csv(predicted_file_path)

# Load the TSV file with correct answers
df_correct = pd.read_csv(correct_answers_file_path, sep='\t')

# Extract the first 100 rows from both DataFrames
df_predicted_100 = df_predicted.head(100)
df_correct_100 = df_correct[['Correct_Answer']].head(100)

# Ensure the DataFrames have the same number of rows
if len(df_predicted_100) != len(df_correct_100):
    raise ValueError("The number of rows in the two datasets does not match.")

# Add the Correct_Answer column to the predicted DataFrame
df_predicted_100['Correct Answer'] = df_correct_100['Correct_Answer'].values

# Calculate the 'Is Correct' column
df_predicted_100['Is Correct'] = df_predicted_100.apply(
    lambda row: row['Predicted Answer'].upper() == row['Correct Answer'].upper(), axis=1
)

# Select and reorder columns for the final output
df_final = df_predicted_100[['Question', 'Predicted Answer', 'Correct Answer', 'Is Correct']]

# Save the final DataFrame to a CSV file
df_final.to_csv(output_file_path, index=False)

print(f"Combined file saved to: {output_file_path}")


Combined file saved to: /content/drive/MyDrive/combined_results.csv


<ipython-input-21-f3eba22193fc>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predicted_100['Correct Answer'] = df_correct_100['Correct_Answer'].values
<ipython-input-21-f3eba22193fc>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predicted_100['Is Correct'] = df_predicted_100.apply(


In [ ]:
import pandas as pd

# Define the path to the combined file
combined_file_path = '/content/drive/MyDrive/combined_results.csv'

# Load the combined CSV file
df_combined = pd.read_csv(combined_file_path)

# Calculate the accuracy
total_predictions = len(df_combined)
correct_predictions = df_combined['Is Correct'].sum()
accuracy = correct_predictions / total_predictions * 100  # Accuracy in percentage

print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 68.00%


***gemini evaluation on my dataset***


*   Accuracy: 68.00%
